In [20]:
from arcgis.gis import GIS
from arcgis.geocoding import geocode
from arcgis import geometry
import arcgis.network as network

import pandas as pd

gis = GIS()

items = gis.content.search("san bernardino forest hiking trails",max_items=3)  

#sign into GIS

USERNAME = 'rpanczer'
PASSWORD = 'Bp62190$'
gis = GIS('https://www.arcgis.com', USERNAME, PASSWORD)



In [21]:
old_address = "Nazareth, PA 18064"
ranger_station = "Skyforest, CA 92385"
town_map = gis.map(old_address, zoomlevel=13)
display(town_map)



# ESRI Python API Sandbox
## Find hiking trails in the San Bernardino Forest
***
This notebook uses ESRI's *GIS*, *Geocoding*, *Geometry* and *Network* modules to calculate the driving time from the town where I grew up in Pennsylvania to a hiking trail head in the San Bernardino Forest.
<br>

```
First we need to use the Geocoding module to geocode Nazareth and add draw the town's location on a map.```

In [22]:
#plot my old town's location on the above map
town_loc = geocode(old_address)[0]
ranger_loc = geocode(ranger_station)[0]
sym = {"color": [255,255,255,255], 
       "size": 3, 
       "type": "esriSMS", 
       "style": "esriSMSCircle"}
popup = { 
    "title" : "My town",
    "address" : town_loc['address']
    }
town_map.draw(town_loc['location'], sym)

```
The next step is to load a map of the San Bernardino area. I zoomed out a bit to include portions of San Bernardino and the surrounding towns.```

In [26]:
# If I visit, I want to explore the area and find a place to hike. San Bernardino National Forest looks good
sb_map = gis.map("san bernardino",zoomlevel=10)
display(sb_map)

```
I then found a USFS Trails layer that includes some of the trails I may want to hike. We will want to add this to the San Bernardino map. We will also want to add the Ranger Station address so we can collect some park information before we start the hike.``` 

In [24]:
# Add hiking trail layer from Vision2BActive to sb_map
sb_map_layers = [temp_item for temp_item in items if temp_item.title == "Vision2BActive_USFS_Trails"]
sb_map.add_layer(sb_map_layers[0])
sb_map.draw(town_loc['location'])
sb_map.draw(ranger_loc['location'],symbol={"angle":0,"xoffset":0,"yoffset":0,"type":"esriPMS","url":"http://static.arcgis.com/images/Symbols/OutdoorRecreation/Hiking.png","imageData":"iVBORw0KGgoAAAANSUhEUgAAAEAAAABACAYAAACqaXHeAAAAAXNSR0IArs4c6QAAAARnQU1BAACxjwv8YQUAAAAgY0hSTQAAeiYAAICEAAD6AAAAgOgAAHUwAADqYAAAOpgAABdwnLpRPAAAGqhJREFUeF7NWwdYVVfWfTSlqEgVBBVQREWpItI7KEUBBSsKIqioCKgUBRUECzYQRUBQwY50LIAGu1FjYomml0kyk0kyk14mmczM+vc+94HEWBIl5r/ft7773rv3lL3O3vvss895Mtmfc/WlZo0ISn9O839eq5HUdCvhM8K3hDuEpQTlP69Lz6/lldQU1Hoqw9RAE8MH6kC7tyr4N8Ipgvrz60r3t6T4hCptWVADbQ1Eeg5DdMBIxBBm+lpilKluBwlbu79bz6fGaGrmFqGM0PMRTe5RUlRAiNNgIfw07+ECM3xGIMp3OIx0ezMJ/yT0eT5d7r5WDKmqHxSUFDpGcfxDqu5Bv73dT0sDUX6WncJP9RqGiWMHoTwnHrPD/bj8vwlm3de151PTFlbtYX6DoKKqzEKwFjx4adMPX5sba3WOPgsf5T8KydM80bInF+HjPbjsJwTN59Pt7mnFnEdfb7Amwje4Qp/u9P0rAk9vXa9eLBw7vtn+kvpHuJtjQdhYtB/YhOv1OxHs48RleWbQ6p6uPZ9aWmUKMngvsMKMQg94xFp2mEHzQ5o/3lutB8JdBiPSw1yofkygLS5UF+C1M/swbYI3l/2JYPJ8uv7srWSz6o/wNMasAndMXeeMqG3uGO5u1EHCgS7qPIw+H+P3ZwU5ImmKKxZFuKJqYxKu1hfj5eYSOFhZcLk3CI9yos/e426sIY2F6T9MC7O2uGI2ISrfRdyjt7nBwtmwg4R79N4uwvf8PiM/LR63TpThck0RrjXswp1TFTh/tAD6Olr8vLob+/iHVbWWBTEc2hezN7sgboc7Yre5ILbAle6uiNvuhoXlXnCJHNJBAhQHj4YsNAOjE/JotMvxUs1WvFS3HW+0lqMoax5GmHVqzcY/rNfdULEa1bGPhR9gqS2ETij1wPwiVyzY4daJxWUemLfdFQOG9xUE9HGJhn7WZSiknYBX+SW8+cEHuHPjAl555TrqW06jl4U7ZOq66KEiZpAfCGxa/y+vcBZolGd/LNnjBRZ0UbEbEkvcsaRUQnK5B1IqPGEyUqgz+njEwzj7JowzLqJf6hloLKmHbkodBqc3QyuqAAruCUjccphI+QSv3buLwMDADq0JeQYGPKmswjOUf2TRGBbK3EEf07LskbybhN3tjqUk9LIKDyzf44Gsoz5wmThICNHbaSaMVlyGwcJaGCyqQ//EJhgtbYPB8rPoEVMBmWcittW0o+v15ZdfwsDAgMvfekoBdsh9ze6nLP/IYtb05G8qPZRgOkobM7PskF7phbS9Hsio9MSKKk9kHfbG4kInKCvJoGJsB6P089CbtQvak9ZDO2IjdCI3wXBxAzQSDkAWlgWVCel4/5PPf0EAf1m3bl2HFgRTm+MI5YRkgvEThMpk4XX0OxdXpd1JAi9XMcrNUIx4dq2fEDpzvydWH/TCKkJRWwBmpY8SndeamA3jrGswSGyEYdJxGJDg/Zc0Q3fpCciiCyCbnANZUBpefvPDXxHw3nvvQVVVCPEd19Wjp1IHIV/IyRj9EMGm8LtWDjqoOuML72DjjjLdQgLPyzPlqgUV6pClUz8sL3VFXo0Pcqu9kXfMG813QxCXNkI0rDutQBBglHYO/VPb0Z/uA9IvoNeSYxIBkbmQeS/GC9df/RUB/IO1tS21I8PUeeYoqnbDsvW2cHDThyItpuT9qKO7lZwIG7p/b2CsjtJGT9RcHYcjF/zhEdg5szw1CexIMghvcQPcsIZmD1i59kPMKlsh9PoaSfjiFj+0vzMR8cuHiQ7qzS6lUa+HXtQuQrG4GyWfhEZSjUTAFFJzx2jUnbn8UALCQiOgqCLDwXN+OHo5ANVXAsR9fcVY+IUNgEoPRW6H+8Rq/05PVSXk7R6LYy+Ow95WHxw464cjF39BAgdlvzvzxNEbtA3V4T7ZDBEpVlha4ooNDX7Ib/LDhlpvbD/ui33n/FFzfRzOvD3hPgGziqE/Zw90Jm8Qts/3/mQGvZPr7xNgPwNFhxp/QcB///0vnDreCGPjgVDvpYTyE17Y305ttPmg8rSPEKr22njklTli0BCxfEYfrR5Iy7cTI8/vCchJOEyaEBA+sENrGun935Vw0acCn2oZqCOhwBn5reORfdQba496CeE3N/iIka88H4Da6+N/ScDsEujH7YdORD50pmwRd8OFdeib0nSfANupSNmwSxDwn3//iHdvnsO905VwHGUuOrxolRWOXgpA5RlJ+K6oJm0oa/IUghccdhXvdQovJ2v/C77g907cCcb0+UPJlwituUDghZoGgZfxvEh77OVAT7/pqaaMifOHI4fsndV+U70PtjX5ouikLypJA+pvjMdp0oC4ZR0m8BACFlRDe+lxyGLYBPIgs5uK6PQN+PTjv+LWmSM4u389hcilCPV1EwTsrPMQDq3ilHcndpNGlB33ktS8XTIPNhMm5yCpPZsKa8Jh0hR+N3+fM9I32WFh5ijo91fv0IR3qf7NhMlyEh5JgAo9WUX4Fzs+r0gzGn1vbKzzwZZGH5Sepg6QzTW+PF4QwCawJFuaBXSmboF+/IFfaIBB/CHoLjtFBBRKBNhPg9/cDFx/8TIu1uzEJVoV3jlVjthIKRiau9wSWUUOSN9ijzRCKgmStd0Bm/e7EAHektAkvHB6JHBRtTtS8mwQMsMEVmN0oGeg1uErOgR/k+pdQxhD4Kj2sRfbyklFyvJYjNHHwm1OWEe2n1stqX8h2f7Bi5Lqs/AdBKTl24jGtMnm9eMP/oIA9gn9lrcRAUUSAe6xUBoZhOS123C1qQKXjxUKAlJiI0Ud0xMskEkCL99oJ4TfRqrOQrODY9tmwXfUuCM6aRiG21HkqSwtth6Cl+i3Gb9F6K6MuHFFAyz6IpHC3dxGf6ylKY/tf2OdN6m/D3ae8sUBIqFBrgFtb4Qgu9hBdKBvcBb6JRyjIIicIPuByRuhN3MnDJefhlLsLokA/0TILIMgG+SBnZtW42ZzsVgZrkmKFnVExg3Bml2OSN9qjwzC2lJHbDnoiooWav+QK8ZHDoKymrBrKMsUMdpcH/OCRsJ2sB4UFMR0+QGBo9enCos5hXVbgeZdVXVlmIzQQkTSSDHnswlsJRNgB7j/Ak1RNBKsAadeC6GOuYgOaThFw3jlFRECGyysEXdDCoSM09qhNm8fEUBxgP8SyKwnQDbAHZkZy3DnRAlun9yNzSvmizrCos2QWzYWKwtGC2RsGy0ICZ1lBmVVSfBhxtpYGGqNXUleaFobIhItCpSgoWccJ7ATf+qLveOLzKSmripGBxgjfr2D8AHsBHkGYDPYc9Yfx65JBDTfDiL19EUvNRkU9SxhmNICo4wLhIvSnUJjY7rrJjdLGuAVD5lVCGTGlEOYNR13T5WRE9yNbVkLhXDB002wrsJJ+AFGHn0Om20mnulrqiNzliNqSeiG3BA05k5A0iTbDvXnOf+ZL2ECOkYamLzcGplHfLC+6b4ZsBM8fGUcmm8GdjrBejKFVjIDZ2896ogitEKyKBbYK2FupQTyA/xdJSgVsjFk6zahkJm4Y4iFBa6QD7h9shyleSlCEP9JA7CBvPjqnWOQQ+rPfkCFRt5ItxcqM0njNofjaHYwDq8ORPWaIAwfpM3lPiL0fmbpqQJNwnnCf5WUFaFDU4ilkz6iKQJcRyYgOUKaCdpo6iET6PADTEAuBSksgMoAO+iS3etMKyQ/QMEQ+QJpUbQZfcNzoUCrRZldOGQjxkGmoIYNy2Lx+plKNJbRM5kSbF10sKnKBdmk9hsrXcgpDhX1Jk0ZjfaiKThGI1+dE4Iauu/JCICmRk9+fqg7hO9aB68BvhHaQCTM3+QoYoHVh7ywjkLgstNSFNhBQIcWjKPR4zLKxjboMz4dfaYUQntGMXRnldIKkUDhsbL7XMgsfCDrIaXP8uLHilTZi3U7YWpkTKanLMJbBju/SXOkDNO6Be5oK4xA3fqJqMmbgFrC4dVBtOMk5vmr3UkAmwHn9GDtbYTkMnesPORNq0APFJD9H6OQlE2gQT4NdkyHra+HiOlJUVFGI0kOSUEJ2oOGQsnIGjIdmiZ1CZocMPUhKMDfRgPzA/VQmWyC0xUZFA3uR0SgpxA2Jc8W+aQFW4mA2UnDxW/JUx1woWQ66jeEEgmhqF03Eae2TMIkT0lDCBy7PPPFO7g/qvZSwbj5lkje54UkWgYnEQkZ+zw7w+DDl7uMvpyI029NQHIuCUudiQ/QR8NKU7y2awgaVvVH0sReiPJSxaLg3siZpY9DGab47Igt3t9jhSNLjXE4OxCvtlUhP32eKB8ROwSFR92IBMkXqFA4a0J7Cye3RaBteyQaNoYJIhrpfowcoY05+x9BwlHCkKdlYS5X0kdPDaHp9lhQ6ol4yvEl7HTD4hI3ygJ5iICoop0WJi9JM0BXMAGR8ZK67kk0xY8N9vi8xhbfnbTHz2ft8V2rHf7VYo9vmu3wD/r9vX1WeL/CCjc2D8HBTDfcoKRp9c41Qjs8ggxRdMwdG8kZbqf7hJmmol6nUUY4QSS075yKxk3hqN8YihPkFOtIG0Jo30FOwtd0zyc8uEnzWF4m0NP/aRn3RthqR8wt9UZskQfiiz0QV+hKCVApB8jzf8urgb8SXqwJiIAICmK4E8ezhuKralt8fNgGn9Ta4Ic2e7JXM1TRyH/VZIf391vh3b1WeHv3KNzdPgxHM8eIXaJ9W1aI8uMiBoo1ATvDzQdcBBle8mSH+QBtVGQG4mLpDDSTCbA2NBMZbQUR2EB+woYCIzkRvN2WSnhi+MvMXFGmtJd1yGB4zhsFl5nDMDrUDGZ2+pi80h7zd7oLIhIpAVpFJBy/9WsSmIA5SyV7PUkEfHn0PgHf0ui3bhkKZ8te+NtRG3x00LqTgFeLRqAyyRw36rdj5cLZonxcqiWK6z2whYRnAjgc3knfOUji5z2UlLAixgkXymbgxNbJQhuYiJZtk3GcPmdFO8FioJgeGbcInk8yi2x5JNVRSNzV+vTAjA3OiKG8P+f+5xBYGyrPBaDp5i9NgBdFC7KkRVF16mB8XW3XqQH/aLIRJuBm1RvLIgzwTaMd3pFrwGs7hqN+1Wi0liRiabjk0CbFDEZpsye2UujbgYIjbmJFmJxrg756YupD7AQrXCQS2Dc0bSZtyA8TfqGViGggH5EQZoPe6j343Z8JS55Egi+9EEEIJTRxAw6ThiCKtr+mb3TGrE0uRIaL8AeHaB3e9MqvfcDyfCkq27vYFN/W3Cfg00Yb/HTGHrU5g6FKWZ3rNOqfkHm8WToKb+wahVvbLHE+ewiu5AyFrroSBg3TFCbAjpAF7womgX3DEEuxIYs4EvLSbtIEEppNQvINEhGnadosWe7bVRuWP4kEfs57+u9rGmogLMcZk3OdEZnrhAgCO8KmVwLRcjdIENAZB5AP4EXRut1SMFQQOxA/1tvj70ds8FmdDb4+ZYf/nB2NpMn9JBsfrYnPyUd8dsgGH1RY48Nya3xaboPiOUbQpIWOAgVhvB5gM9hOucGuKKTvZc1e4LvpMJ5SZdiS5C2ZA2nCcTIJ1oZG0oY60oIW+l5NofNIs87TKBOfREIlL4bGzByBiXmuCFk1FqFrKB7PdhKbILwe2NXqh4OkBez8TpA/OHknEO1kAmuKx4gOLQ8zwA91dvim3g5fNtnikwZrpE4Xef9OBI/RRFWKCWrSzLB7gRFifXQwyoK9vRIMB6hjG408E8CxxYMoot/KT3pjXflYKNMUOYQWR+27pqOVIsWT5AyZhGYmQT5TnKLvByho0uurxu3/laD5KBLsuJP6w7Thv9IJASvHYvwKRwRljsUEIiI4a6z4HEpkTF3vjLnkGJeWuyOPokReN7jOkGy4p4oCRg1UgcvwHrAxU6YDUQ9br/PSVQl9eqnDYogpZk8JRUV+Onoqq4lweB/lANgMHof9lB3ynyTl/gqW+eFSRRRamITCSKENbBJNcgfZvmMKUqbRXqU0CLzf8NBrBL/AJz40dNSgZ64F90Q7BK5xRnC2MybmuiAsj7DOlbTDBYE5LhhHz8atdkLIWhf4JNpCScrcwsLcHNYjR2K0jTW8XBwxfaIfBhlLJsAwMtDF6qQY5KXOQ1FOElqq8nGNtswHGfSHkZmqcHgljZ7Y1eDxSHCGKKtQEiomxArXq6LRRjECa8IpCpi6asNxMWWGikUVvX+D8Mh8QRA9/FII4WsCrxQHOMZawSbCAsMDTDHIwQCGljqw9x6AsOkWCI4ZgWA6KOFPwo8jjTG2laKyVZTgqNySgZL1y1BbkoPTBzbD3ERsXNwl1CopKiI9YQZK6fmudUvFtvktygs4Wlmih7pMqD3n95iEEkqEdkUpf6ffqygBmi03u9nBVnj5YAxeoHD5dPE0tO2Y+ittOEu/T3ATsQofyDB9nC94XZm0oO+A3tQZFbBP6GK/nJf/n4maAiabqWGCiSrCzTXgPEANyvSuioYK7fiqYO2yuShck4jDRavEYqehbC169lDhengXWIc74TnWVghfvjGVCNiOOy0ViBgvrQc27HESSVB2eA+Cp0jOBnMG2Hqs5Nz2ZIfg+oEYnN09E+0UJJ0hn/CgNpwnclKmS9krQsDjCGBPySssTidXEXihMY3gSJhE+K/XMF0sCxiMxd6mWOw1CHOcjKFO3psrNzbUQ8Hqxdi5NlkcgniVBNuQFtfRME+1fN0ZYmIk1L+EcgEXjhaK9zpyg2mUB+BEC2vBgzh03o+coBfGeksmFT3BGrePxePi3tk4T37gHJNAs8KD2nCeiMmjaFFOAG+rPdUldouXhlujaqkPShZ5oGQxjUiKNzk8aTS8nGhkSaj921aIZS4LNjXEi5/xcTgTeavtTNS2VYuwPTsRbfs34V7rHuQkzxF18MLq0Hl/4e07wOlyzvmz2g8cImwZkQEjcLsmHtcPxeIy+YBL+2bjwp5ZOF8e9SttuEzk5MyX0u8ETo8/1VWlqqKEzCl22Bzr1IldCW7wsuovKk+ICkUJqXZbFZ0EayimA1GFGEDC0rMrXVqs7tunF/JXLBDacnzPerzWtreTgKXrbUTam/cH2NZ5+2tHrTvGTe7c8UH6XBe8eXwhbtLoXz04B1cPzMGV/TG4XBn9UG24Rr8vnto5E7g+lfRUaIsi5QzV6dyvBkGHzvzq9FGFrqYqRXlK6E3TWl5qHA4UZNLo7xCObTsJKGd9WZdGt/LpkOyUOeQrFqO+dK0gYEUCZYyIRN4YraP0O6fBSxo8ETl3CHpqSCbmYGmEWprq/nImCbfq5uNGdRxeOjIX1w/HCiJe7EICa8M51gYyiev7o+HtYMJ1fE7gBPBTXXykdQ2Bd1+PEPiAE6Oe8JPlUFMU56agpTJfEMA5/2BvcRbwR0JXz5vEwqTERWIH+YGjO1YLApZETxJCri93Ep4+NMoMar2lrXKT/lrYlBqAd1qXCOHvNCzArfr5uFk7D6+QFtw4GkckzMU1MocXWRvkJnFx7yxcpc+NFBv0ktYGtU8l+RMKGTAB/u4OOFCYKbz6NVL/ViKCVZ2enXygfCgLFTs1CMXkLw4UrBQEpMRGCGEt7Wk1R4cu+LOZkTY2pQXgrbYl+PRKKt4iAl47sQj3mhfi1cYE3K4nIkgTXqmZh5cfog2XSPXv1c1D/P0McuAfQYAndzYiyAv1NOVdJefHo5+dLG14EKIeaJSjzv+F+rsKh7l3czrukhOcN4NS5vJgydmWzhOvC8Vfzi3FVy+vxAd0f/t0kiDgjVOJeP3kYtw7vgh3mxKENtx+iDawSdwhYhoKIjtG/9IfITzX6ccdjwqnNYKcALZ/Hxc7FohPefDc3/ViG/zC0XYEESDFAq/SDlH0JMoW87y+IRw/3FuD7+6swsdX0vDB+WWCiPfaU/DOmWS8TdrwZksiEbH4kdrAvuEWmccdwpiRwkH/h8DE/yEXL6eJAH8cIXu+TochT9PUpqslVmy8Y/PgxaHoDRNjAxRQwFRGESHvECXFTBYEXKFR+/H1bHz8Yhr+RgR8dCkVH16QSHj/bArefSEZ7zxGG9gk7pBpvEskTQ+S8hSE9D9EcnmlnRqwj0Jgtv9jlONTVhbnAB91BvBgL3U1bMlcSDHDSrx3/iA2ZUiJ0ZxkX+CD9fju7mr8/Wq6IOKvl1Px0cXlT9SGu+QbXifT+LA9GYkzpSU6oVt2jh5HoAs3FBHkibINy0Rkd2znaqhIBDwqZZ2mROmt8Z6OCPRyRMKMiXAbPbLTB0z0HYaL1fH4+a21+PJmJj4nP/DFK5n49vYqfHMrC1/R50+JHNaG+75hMd6jGeJDMpV5tJkiF57/lvOH/zeJDy/9NyzAjZa1abhSW4SqrRkdHUh4BHMT5R2kFagKemv1ouNufaGnr4XBQ6WAR0tTjQKqOXj77FK8cToZF47EoXx9GHJTfLFhuT9aKfz950sZnQ7yE5ol2EGG0r9R5HXzYe3flBh9VvPg83zfj7EZjvWp8WjZt5GCmhkdnah5ROXiFKqn3xjELAjHguQpiF8Sgai4CdhaloopsySHyJu1fXr1RB8KupTl640O4pSVFMlvOOOLl1fg+9tZaCNCRg3tXHZvp/ee9J+mZ5W7szxvSPykrq4Ks8FG0JGcX1fspe8PntjihRH8g10QnxiJWfETBaIXhCFl5WxMn0PnCKQ63iE0EA4ScghMHHtzT8JxfidxNoXlGeOgQf9NoO981nBet0n2Gyvi0xjw8HPAghQ6B7QgFObDB8HR1QpjnDvtmt/peondaCc3a8xbMgWz54UK4ecunow5CeHw8HXoOPSw5jF9YFLFfxLk4FWs/W/sc7e+tlOFYvvIqHGdQvBoxiVGYEl6FNTURCr7wf8V9aPffjQljWGBmYBpMUFw9xkNg/66HcJ/Re/w+Z7HXXyuiUliZ8tJ3T/latHs2xtRcycgen6YsGO+zye7jpgRACaHelX4kJ4d5pHrq90HuuT85ETxu7znzzZs8adI8xSNnuzTt5cYxRhS4zkJkwQBrl72Qji5enK67cGLT6XwnzE4+cJhKhPC9vtg5PgUXXq+RXYoKSshKMyDbDgCU6MDYTyo0xuzE1v4fLvz/Fvjw5U/s6r3M9SBmrq0fUXIJXTL8ZXnL9Lvb5GzLBxyXiPw9BT2+6t4fiX+DxsJUta6ulAMAAAAAElFTkSuQmCC","contentType":"image/png","width":24,"height":24});

In [25]:
route_service_url = gis.properties.helperServices.route.url
route_service = network.RouteLayer(route_service_url, gis=gis)

stops_json = '{0},{1}; {2},{3}'.format(esri_loc['attributes']['X'],
                                  esri_loc['attributes']['Y'],
                                  ranger_loc['attributes']['X'],
                                  ranger_loc['attributes']['Y'])
driving_route = route_service.solve(stops=stops_json, returnDirections=True, returnRoutes=True, 
                           outputLines='esriNAOutputLineNone', returnBarriers=False, 
                           returnPolygonBarriers=False, returnPolylineBarriers=False)

travel_time = driving_route['routes']['features'][0]['attributes']['Total_TravelTime']
print("Time to drive to the hiking parking lot is about {0:.1f} minutes.".format(travel_time))


RuntimeError: User does not have permissions to access 'world/route.mapserver'.
(Error Code: 403)